##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Visualización de datos mediante el Proyector de incorporaciones de TensorBoard

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tensorboard/tensorboard_projector_plugin.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tensorboard/tensorboard_projector_plugin.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tensorboard/tensorboard_projector_plugin.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

## Descripción general

Utilizando el **Proyector de incorporaciones de TensorBoard**, puede representar gráficamente incorporaciones de alta dimensión. Esto puede ser útil para visualizar, examinar y comprender sus capas de incorporación.

En este tutorial, aprenderá a visualizar este tipo de capa entrenada.

## Preparación

En este tutorial, utilizaremos TensorBoard para visualizar una capa de incorporación generada para clasificar datos de reseñas de películas.

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

%load_ext tensorboard

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector


## Datos de IMDB

Utilizaremos un conjunto de datos de 25,000 críticas de películas de IMDB, cada una de las cuales tiene una etiqueta de sentimiento (positivo/negativo). Cada crítica se preprocesa y se codifica como una secuencia de índices de palabras (números enteros). Para simplificar, las palabras se indizan por frecuencia global en el conjunto de datos; por ejemplo, el entero "3" codifica la 3ª palabra más frecuente que aparece en todas las reseñas. Esto permite realizar rápidamente operaciones de filtrado como: "considerar sólo las 10,000 palabras más frecuentes, pero eliminando las 20 palabras más frecuentes".

Como convención, "0" no representa ninguna palabra específica, sino que se utiliza para codificar cualquier palabra desconocida. Más adelante en el tutorial, eliminaremos la fila para el "0" en la visualización.


In [ ]:
(train_data, test_data), info = tfds.load(
    "imdb_reviews/subwords8k",
    split=(tfds.Split.TRAIN, tfds.Split.TEST),
    with_info=True,
    as_supervised=True,
)
encoder = info.features["text"].encoder

# Shuffle and pad the data.
train_batches = train_data.shuffle(1000).padded_batch(
    10, padded_shapes=((None,), ())
)
test_batches = test_data.shuffle(1000).padded_batch(
    10, padded_shapes=((None,), ())
)
train_batch, train_labels = next(iter(train_batches))


# Capa de incorporación de Keras

Una [Capa de incorporación de Keras](https://keras.io/layers/embeddings/) puede utilizarse para entrenar una incorporación para cada palabra de su vocabulario. Cada palabra (o subpalabra en este caso) se asociará a un vector de 16 dimensiones (o incorporación) que será entrenado por el modelo.

Consulte [este tutorial](https://www.tensorflow.org/tutorials/text/word_embeddings?hl=en) para obtener más información sobre la incorporación de palabras.

In [9]:
# Create an embedding layer.
embedding_dim = 16
embedding = tf.keras.layers.Embedding(encoder.vocab_size, embedding_dim)
# Configure the embedding layer as part of a keras model.
model = tf.keras.Sequential(
    [
        embedding, # The embedding layer should be the first layer in a model.
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1),
    ]
)

# Compile model.
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Train model for one epoch.
history = model.fit(
    train_batches, epochs=1, validation_data=test_batches, validation_steps=20
)

2500/2500 [==============================] - 13s 5ms/step - loss: 0.5330 - accuracy: 0.6769 - val_loss: 0.4043 - val_accuracy: 0.7800


## Guardando datos para TensorBoard

TensorBoard lee tensores y metadatos de los registros de sus proyectos de tensorflow. La ruta al directorio de registros se especifica a continuación con `log_dir`. Para este tutorial, utilizaremos `/logs/imdb-example/`.

Para cargar los datos en Tensorboard, necesitamos guardar un punto de verificación del entrenamiento en ese directorio, junto con metadatos que permitan visualizar una capa específica de interés en el modelo. 

In [ ]:
# Set up a logs directory, so Tensorboard knows where to look for files.
log_dir='/logs/imdb-example/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for subwords in encoder.subwords:
    f.write("{}\n".format(subwords))
  # Fill in the rest of the labels with "unknown".
  for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
    f.write("unknown #{}\n".format(unknown))


# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(model.layers[0].get_weights()[0][1:])
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [ ]:
# Now run tensorboard against on log data we just saved.
%tensorboard --logdir /logs/imdb-example/

<!-- <img class="tfo-display-only-on-site" src="images/embedding_projector.png?raw=1"/> -->

## Análisis

El proyector TensorBoard es una gran herramienta para interpretar y visualizar la incorporación. El panel de control permite a los usuarios buscar términos específicos y resalta las palabras que son adyacentes entre sí en el espacio de incorporación (de baja dimensión). En este ejemplo podemos ver que Wes **Anderson** y Alfred **Hitchcock** son ambos términos bastante neutros, pero que se hace referencia a ellos en contextos diferentes.

<!-- <img class="tfo-display-only-on-site" src="images/embedding_projector_hitchcock.png?raw=1"/> -->

En este espacio, Hitchcock está más cerca de palabras como `nightmare`, lo que probablemente se deba al hecho de que se le conoce como el "Maestro del suspenso", mientras que Anderson está más cerca de la palabra `heart`, que es coherente con su estilo implacablemente detallista y conmovedor.

<!-- <img class="tfo-display-only-on-site" src="images/embedding_projector_anderson.png?raw=1"/> -->